<a href="https://colab.research.google.com/github/danijak/testing/blob/master/Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [3]:
# loading the MNIST dataset from keras
data = keras.datasets.mnist 

In [47]:
# training & testing split
(train_images, train_labels), (test_images, test_labels) = data.load_data()

In [48]:
# Input Normalize to get it into 0-1
train_images = train_images/255.0
test_images = test_images/255.0


In [51]:
# model
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(28,28)))
model.add(keras.layers.Reshape(target_shape=(28,28,1)))
model.add(keras.layers.Conv2D(12,kernel_size=(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10))
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 26, 26, 12)        120       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 13, 13, 12)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 2028)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                20290     
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
_________________________________________________________________


In [55]:
#Train the model
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(x=train_images, y=train_labels, batch_size=32, epochs=5, validation_split=0.1)

Epoch 1/5
1688/1688 [==============================] - 18s 11ms/step - loss: 0.2577 - accuracy: 0.9290 - val_loss: 0.1089 - val_accuracy: 0.9702
Epoch 2/5
1688/1688 [==============================] - 18s 10ms/step - loss: 0.1043 - accuracy: 0.9703 - val_loss: 0.0768 - val_accuracy: 0.9788
Epoch 3/5
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0791 - accuracy: 0.9765 - val_loss: 0.0706 - val_accuracy: 0.9810
Epoch 4/5
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0666 - accuracy: 0.9803 - val_loss: 0.0669 - val_accuracy: 0.9832
Epoch 5/5
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0577 - accuracy: 0.9827 - val_loss: 0.0636 - val_accuracy: 0.9845


In [56]:
# Evaluate
print(model.evaluate(x=test_images, y=test_labels))


313/313 [==============================] - 2s 5ms/step - loss: 0.0657 - accuracy: 0.9793
[0.0657241940498352, 0.9793000221252441]


In [57]:
! pip install tensorflow_model_optimization

     |████████████████████████████████| 174kB 2.8MB/s 


In [73]:
# Pre trained model with pruning
import tensorflow_model_optimization as tfmot
num_images = train_images.shape[0] * 0.9
end_step = np.ceil(num_images / 128).astype(np.int32) * 2

# Model for pruning
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}
model_for_pruning=tfmot.sparsity.keras.prune_low_magnitude(model,**pruning_params)


# Compile model
model_for_pruning.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer='adam',
      metrics=['accuracy']
)
model_for_pruning.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape_ (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten_ (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense_15 (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________


In [74]:
# fitting 
model_for_pruning.fit(train_images, train_labels,batch_size=128, epochs=2, validation_split=0.1, callbacks=tfmot.sparsity.keras.UpdatePruningStep())

Epoch 1/2
422/422 [==============================] - 13s 32ms/step - loss: 0.0781 - accuracy: 0.9782 - val_loss: 0.0731 - val_accuracy: 0.9793
Epoch 2/2
422/422 [==============================] - 13s 32ms/step - loss: 0.0743 - accuracy: 0.9786 - val_loss: 0.0737 - val_accuracy: 0.9803


In [75]:
print(model_for_pruning.evaluate(x=test_images, y=test_labels))

313/313 [==============================] - 2s 5ms/step - loss: 0.0832 - accuracy: 0.9767
[0.08324266970157623, 0.9767000079154968]
